# Setup - Download DINO ViT-S/16 and CIFAR-100

This notebook downloads:
1. **DINO ViT-S/16** model from the official Facebook Research repository
2. **CIFAR-100** dataset using torchvision

## 1. Install Required Dependencies

In [ ]:
# Install required packages (run if needed)
#!pip install -q -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [23]:
import torch
import torchvision
import torchvision.transforms as transforms

print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.9.0+cpu
Torchvision version: 0.24.0+cpu
CUDA available: False


## 2. Download DINO ViT-S/16 Model

Loading the pretrained DINO ViT-S/16 model from the official Facebook Research GitHub repository:
https://github.com/facebookresearch/dino

In [24]:
# Load DINO ViT-S/16 pretrained model from Facebook Research repository
print("Downloading DINO ViT-S/16 model...")

dino_vits16 = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')

# Set model to evaluation mode
dino_vits16.eval()

print("\nDINO ViT-S/16 model loaded successfully!")
print(f"Model architecture:\n{dino_vits16}")

Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main



DINO ViT-S/16 model loaded successfully!
Model architecture:
VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0-11): 12 x Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (norm): LayerNorm((384,),

In [25]:
# Model information
total_params = sum(p.numel() for p in dino_vits16.parameters())
trainable_params = sum(p.numel() for p in dino_vits16.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

Total parameters: 21,665,664
Trainable parameters: 21,665,664


## 3. Download CIFAR-100 Dataset

Downloading the CIFAR-100 dataset using torchvision. The dataset contains 60,000 32x32 color images in 100 classes.

In [26]:
# Define the data directory
data_dir = './data'
os.makedirs(data_dir, exist_ok=True)

print(f"Data will be saved to: {os.path.abspath(data_dir)}")

Data will be saved to: /content/data


In [27]:
# Download CIFAR-100 training set
print("Downloading CIFAR-100 training set...")
cifar100_train = torchvision.datasets.CIFAR100(
    root=data_dir,
    train=True,
    download=True
)
print(f"Training set size: {len(cifar100_train)} images")

# Download CIFAR-100 test set
print("\nDownloading CIFAR-100 test set...")
cifar100_test = torchvision.datasets.CIFAR100(
    root=data_dir,
    train=False,
    download=True
)
print(f"Test set size: {len(cifar100_test)} images")

100%|██████████| 169M/169M [00:03<00:00, 49.1MB/s] 


Training set size: 50000 images

Test set size: 10000 images


In [28]:
# Display dataset information
print("\n" + "="*50)
print("CIFAR-100 Dataset Information")
print("="*50)
print(f"Number of classes: {len(cifar100_train.classes)}")
print(f"Image size: 32x32x3")
print(f"Training samples: {len(cifar100_train)}")
print(f"Test samples: {len(cifar100_test)}")
print(f"\nSample classes: {cifar100_train.classes[:10]}...")


CIFAR-100 Dataset Information
Number of classes: 100
Image size: 32x32x3
Training samples: 50000
Test samples: 10000

Sample classes: ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle']...


## 4. Summary

Setup completed successfully!

In [ ]:
print("\n" + "="*50)
print("Setup Complete!")
print("="*50)
print("\nReady for next steps:")
print("- DINO model is available as 'dino_vits16'")
print("- CIFAR-100 data is saved in './data' directory")
print("- src imports are working")